In [ ]:
from typing import List

import torch
import torch.nn as nn

from transformers import AutoModelForCausalLM, AutoTokenizer


# можете сменить на mps на макбуке, но лично у меня он криво работает
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Знакомство с Transformers - 35 баллов

## Создание модели и предсказание следующего токена - 5 баллов
Нужно создать модель через `AutoModelForCausalLM`, создать токенайзер через `AutoTokenizer` и олучить следующий токен через жадную генерацию!

**Внимание** на каких-то из функций далее у вас может кончаться видеопамять из-за хранения активаций. Чтобы этого не происходило рекомендуется все вычисления оборачивать в контекстный менеджер `with torch.no_grad()`

In [ ]:
model_name = "openai-community/gpt2"
model = AutoModelForCausalLM # Ваш код здесь
tokenizer = AutoTokenizer # ваш код здесь


text = "This is a sample text"

# Нужно преобразовать text с помощью tokenizer() и подать это в model.forward() (он же просто model())
# после этого мы получим logits [batch_size = 1, seq_len, d_model]
# По этому тензору нужно предсказать следующее слово!

inputs = tokenizer(text, ...)

outputs = model(...)
logits = ...
next_token_idx: int = ...


next_token = tokenizer.decode([next_token_idx])

assert next_token.strip() == "file"



## Используем Generate - 5 баллов

Мы с вами помним про различные виды сэмплинга - top_k, top_p, temperature,frequency penalty.
Отличная новость заключается в том, что нам не нужно все это писать самим! Оно уже включено в [GenerationMixin](https://huggingface.co/docs/transformers/v4.44.2/en/main_classes/text_generation#generation), от которого наследуются модели для генерации текста.

Для генерации есть функция [generate](https://huggingface.co/docs/transformers/v4.44.2/en/main_classes/text_generation#transformers.GenerationMixin.generate)

Ваша задача написать для модели выше генерацию по тексту с:
* Температурой - 0.9
* Top-K - 20
* Repetition Penalty (Frequency Penalty) - 1.2
* максимальное число новых токенов - 10


In [ ]:
text = "This is still a sample text, but"
inputs = tokenizer(text, ...)

results = []
for i in range(10):
    gens = model.generate(
        ...
    )
    genertaion: str = ... # сгенерированный текст
    results.append(generation)

assert len(set(results)) > 1, "Все генерации получились одинаковыми, проверьте опции генерации и флаг do_sample!"

## Generate Batched - 5
Теперь давайте жадно сгенерируем текст, но забатчуем несколько сэмплов. До этого мы всегда генерировали по батчу размера 1, поэтому у нас не было паддингов!

Когда появляется несколько текстов разной длины, то появляются и паддинги.

Представим себе ситуцию, что у нас батч из двух элементов длины 2 и 5 (токен -1 будет выступать в качестве паддинга **только для удобства визуализации**).

Тогда 

```python
input_ids = [
    [3, 2, -1, -1, -1]
    [5, 6,  7,  1,  2]
]
attention_mask = [
    [1, 1, 0, 0, 0],
    [1, 1, 1, 1, 1]
]
```

Представим, что мы сгенерировали еще один токен, тогда

```python
input_ids = [
    [3, 2, -1, -1, -1, 7]
    [5, 6,  7,  1,  2, 8]
]
attention_mask = [
    [1, 1, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1]
]
```

Получается, что у нас паддинги в маске возникают посередине. Мы не будем заниматься реализацией своего алгоритма генерации здесь, но отметим, что добавление паддинга слева значительно упрощает этот процесс.
Тогда исходная последовательность будет:

```python
input_ids = [
    [-1, -1, -1, 3, 2]
    [ 5,  6,  7, 1, 2]
]
attention_mask = [
    [0, 0, 0, 1, 1],
    [1, 1, 1, 1, 1]
]
```

и после генерации следующего токена

```python
input_ids = [
    [-1, -1, -1, 3, 2, 7]
    [ 5,  6,  7, 1, 2, 8]
]
attention_mask = [
    [0, 0, 0, 1, 1, 1],
    [1, 1, 1, 1, 1, 1]
]
```

В качестве задания давайте соберем батч с левым паддингом и проверим, что жадная генерация (10 токенов) совпадает с генерацией на текстах по отдельности!

Для этого нам придется использовать параметр padding_side в конструкторе токенизатора.

In [ ]:
tokenizer = AutoTokenizer # ваш код здесь
tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
texts = ["This is a sample text", "I'm really tired and this is just about"]

# Внимание! В данном задании нужна жадная генерация!

# Соберите оба текста в один батч и положите результаты генерации в 
# batched_generations
batched_generations: List[str] = []

....

# Пройдитесь по каждому сэмплу по отдельности и положите результаты генерации 
# в single_generations
single_generations: List[str] = []

...

assert len(batched_generations) == 2 and len(single_generations) == 2
for s, b in zip(batched_generations, single_generations):
    assert s == b

# KV Cache - 10 баллов
При генерации есть опция `use_cache` - это использование KV cache для генерации. Посмотреть визуализации про kv cache можно [тут](https://medium.com/@joaolages/kv-caching-explained-276520203249), или [тут](https://developer.nvidia.com/blog/mastering-llm-techniques-inference-optimization/#key-value_caching), или [тут](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/general/appnotes/transformers-neuronx/generative-llm-inference-with-neuron.html#kv-caching) или в лекции 4.

В рамках этой техники в при генерации в декодере считается только аттеншн последнего токена по всем векторам предыдущих токенов, которые посчитали на предыдущих этапах, а для "старых" (левых) токенов аттеншн не пересчитывается, т.к. "новые" (правые) токены на них не влияют.

Мы сохраняем K,V проекции предыдущих токенов (не пересчитываем $X W_K$ и $X W_V$) и считаем аттеншен только последнего токена по всем предыдущим. Аттеншены для предыдущих токенов мы не считаем, т.к. они для генерации не нужны, их выходы уже сохранениы в KV cache.

В рамках данного задания нужно:
1. Посчитать скорость генерации 100 токенов с и без kv cache, сказать, какая техника и во сколько раз быстрее.
2. Подсчитать скорость генерации 1 токена с и без kv cache, сказать, какая техника быстрее и почему.

Чтобы корректно сравнивать время генерации нужно использовать жадный сэмплинг!

**Ответы на оба вопроса нужно оставить письменно прямо здесь**.

<<Место для ответа>>

In [ ]:
import time
import numpy as np

text = """
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Vestibulum lorem justo, semper dignissim ipsum vitae, sollicitudin aliquet eros. Duis id ultricies erat. Vivamus commodo auctor massa ut mollis. Maecenas lacinia tempus orci, imperdiet ullamcorper felis accumsan et. Etiam mattis neque diam, at egestas nunc eleifend id. Fusce tristique orci nec sollicitudin elementum. Nullam dui est, feugiat ac pellentesque at, posuere non massa.

Suspendisse accumsan ullamcorper dolor sed dictum. Mauris quis varius felis, quis gravida odio. Vestibulum diam arcu, aliquet convallis congue non, rutrum non turpis. Fusce vel orci ac diam suscipit lacinia. Curabitur maximus orci a dui gravida, accumsan convallis libero ornare. Phasellus dapibus, sapien pulvinar lacinia dictum, massa lacus scelerisque tellus, eu porta dolor eros vitae ex. Maecenas maximus, urna id pharetra dictum, dolor lorem sollicitudin ipsum, sit amet vestibulum orci felis quis leo. Pellentesque vel ligula ut urna eleifend condimentum nec et sem. Integer ligula nunc, rutrum ultricies urna et, congue suscipit lectus.
""".strip()

# Подсчитаейте время генерации 100 новых токенов с помощью жадного сэмплирования при включенном KV Cache, выведите среднее время работы
...

# Подсчитаейте время генерации 100 новых токенов с помощью жадного сэмплирования при выключенном KV Cache, выведите среднее время работы
...

# Не забудьте ответить на вопросы вы описании задания!


In [ ]:
text = """
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Vestibulum lorem justo, semper dignissim ipsum vitae, sollicitudin aliquet eros. Duis id ultricies erat. Vivamus commodo auctor massa ut mollis. Maecenas lacinia tempus orci, imperdiet ullamcorper felis accumsan et. Etiam mattis neque diam, at egestas nunc eleifend id. Fusce tristique orci nec sollicitudin elementum. Nullam dui est, feugiat ac pellentesque at, posuere non massa.

Suspendisse accumsan ullamcorper dolor sed dictum. Mauris quis varius felis, quis gravida odio. Vestibulum diam arcu, aliquet convallis congue non, rutrum non turpis. Fusce vel orci ac diam suscipit lacinia. Curabitur maximus orci a dui gravida, accumsan convallis libero ornare. Phasellus dapibus, sapien pulvinar lacinia dictum, massa lacus scelerisque tellus, eu porta dolor eros vitae ex. Maecenas maximus, urna id pharetra dictum, dolor lorem sollicitudin ipsum, sit amet vestibulum orci felis quis leo. Pellentesque vel ligula ut urna eleifend condimentum nec et sem. Integer ligula nunc, rutrum ultricies urna et, congue suscipit lectus.
""".strip()

# Подсчитаейте время генерации 1 нового токена с помощью жадного сэмплирования при включенном KV Cache, выведите среднее время работы
...

# Подсчитаейте время генерации 1 нового токена с помощью жадного сэмплирования при выключенном KV Cache, выведите среднее время работы
...

# Не забудьте ответить на вопросы вы описании задания!


# Скоринг, Perplixity - 6 баллов

Можно не только генерировать текст. Вспомним, что выдает после lm_head - вектор `[batch_size, seq_len, vocab_size]`, где для каждый вектор `[vocab_size]` это распределение вероятностей по следующему токену!

Опустим размерность batch_size=1 для удобства, seq_len = 4. Пусть у нас есть текст `bos мама мыла раму` (`bos` спецсимвол для начала текста)

Тогда вероятность этого текста расписывается через произведение условных вероятностей:

```
P(bos мама мыла раму) = P(мама | bos) * P(мыла | bos мама) * P(раму| bos мама мыла)
```

Т.е. это вероятность слова при условии его левого контекста.
Зачастую ее обозначают как $P(x_i|x_{<i})$ где $x_i$ - i-е слово, $x_{<i}$ - контекст $[x_1, x_2, x_3, ... x_{i-1}]$
Эти вероятности можно взять из выходного вектора!

Давайте попробуем подсчитать вероятность и perplexity текстов!
perplexity как и вероятность мера того насколько модель "уверена" в тексте, т.е. насколько по оценки ее параметрами данный текст вероятен.

$$Perplexity(X) = exp(-\frac {1} {N} \sum_{i}^{N} log P(x_i | x_{<i}))$$

В этом задании нужно:
1. Посчитать вероятность **text**
2. Посчитать перплексию **text**

Еще одна важная деталь:
работать с вероятностями плохо. Т.к. вероятность представляет собой число от 0 до 1, то при перемножении десятков или даже сотен таких числе теряется точность!
Для этого от произведения вероятностей берут логарифм и получают logprobs - логарифмы вероятностей. Их можно складывать, по свойству логарифма логарифм произведения равен произведению логарифма.

$$ p = p_1 * p_2 * p_3 $$
$$log(p) = log (p_1) + log (p_2) + log (p_3)$$
$$exp(log (p)) = p = exp(log (p_1) + log (p_2) + log (p_3)) = exp (log (p_1 * p_2 * p_3)) = p_1 * p_2 * p_3$$

В pytorch для этого есть `torch.log_softmax`, который считается численно стабильно!

In [ ]:
print(f"Beginning of sentence (BOS) token = `{tokenizer.bos_token}`")
print(f"End of sentence (EOS) token  = `{tokenizer.eos_token}`")
text = "<|endoftext|>I'm so very tired of this<|endoftext|>"

inputs = tokenizer(text, ...)



with torch.no_grad():
    logits = model(...).logits
    ...
    # ваш код здесь!
    # 1. Нужно обрезать logits по длине, т.к. для предсказаний по последнему токену нечего считать
    # 2. Превращаем logits в log_probs
    # 3. Берем вероятности следующих токенов, т.к. по вектору i-й позиции мы предсказываем токен на позиции (i + 1)
    # для этого нам поможет torch.gather
    # 4. Считаем вероятности и perplexity!


# должно получиться что-то около 2.1783e-14 для вероятности и около 51 для ppl

# Вопросы - 4 балла

**Ответьте на вопрсоы текстом прямо здесь!**


1. Какое значение P(X) вероятности текста самое "лучшее" в том смысле, что модель максимально уверена в этом тексте и скорее всего его сгенерирует.
2. Какое значение перплексии текста самое "лучшее" в том смысле, что модель максимально уверена в этом тексте и скорее всего его сгенерирует.



# Chat-Models

# Формат - 5 баллов
Как мы обсуждали на лекции, все chat-модели принимают входы в своем особом формате.
Он может быть описан текстом, а может быть заложен в шаблон, который доступен через `tokenizer.apply_chat_template`

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Meta-Llama-3-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("NousResearch/Meta-Llama-3-8B-Instruct", torch_dtype=torch.half).to(device)

In [ ]:
def move_to_device(d):
    for k, v in d.items():
        d[k] = v.to(device)
    return d

Давайте посмотрим, как chat модель отработает на обычном тексте. Используйте для генерации сэмплинг и kv cache, выведите 5 результатов генерации.

In [ ]:
text = "hello how are you"
inputs = tokenizer(text, ...)

for i in range(5):
    # model.generate...
    generated_text = ...
    print(generated_text)
    print("====" * 3)


Видим, что текст зачастую выходит мусорный. Это потому что формат входных данных сильно отличается от того, что модель видела на обучении.
Как мы уже обсуждали, у всех chat-моделей свой формат. Где-то он описан просто словами, где-то он заложен в токенайзер. Мы рассмотрим как раз такой случай - за нас есть удобно написанная функция `apply_chat_template`. Давайте используем ее, чтобы получить префикс для генерации модели.

Не забудьте про опцию add_generation_prefix - она добавляет часть формата, после которой ожидается ответ модели!

In [ ]:
messages = [
    {"role": "user", "content": "hello"}, 
    {"role": "assistant", "content": "I'm good. How can I help you today"},
    {"role": "user", "content": "I love you"},
]

prefix = tokenizer.apply_chat_template(...)

reference = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>

hello<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I'm good. How can I help you today<|eot_id|><|start_header_id|>user<|end_header_id|>

I love you<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

assert prefix.strip() == reference.strip()

Давайте посмотрим, что нам ответит модель!

In [ ]:
inputs = tokenizer(prefix, ...)
model.generate...
print(...)

## Benchmark - 15 баллов

Перед нами датасет MMLU - датасет вопросов и ответов в стиле multiple choice.
* question - вопрос
* choices - варианты ответа
* answer - номер правильного ответа

In [ ]:
from datasets import load_dataset
mmlu = load_dataset("cais/mmlu", "global_facts", split="test")
mmlu[1]

Наша задача здесь решить задачу многоклассовой классификации.
Для этого нужно посчитать 
$$P(choices_i | question)$$
т.е. для посчитать вероятность каждого варианта ответа для вопроса. Мы это уже делали кодом выше!

После этого давайте брать самый вероятный ответ и считать, что модель его выбрала.
После этого давайте посчитаем accurracy, т.е. долю правильных ответов.
Вместо вероятностей для подсчета лучше использовать logprobs.

Итого, что нужно сделать:
1. Пройтись по датасету, для каждого question и каждого из соответствующих choices получить самый вероятный ответ.
2. Посчитать итоговый accuracy

* За базовую реализацию с двойным циклом по questions и choices дается 5 баллов.
* еще 3 балла дается, если собрать question и все его choices в батч размера 4 и считать вероятности параллельно для всего батча сразу.
* еще 3 балла дается, если собрать несколько questions и их choces в батч размера 4 * num_questions_in_batch и считать вероятности параллельно для всего батча сразу.

Что важно помнить:
1. Данные нужно подавать в правильном формате для llama3!
2. Если делаете варианты с батчеванием помните: длины choices могут быть разными! Нужно не считать вероятности по паддингам. В этом нам помогут attention_masks из выходов `tokenizer()`

In [ ]:
# ваш код здесь!

**Ответьте на следующие вопросы (2 балла за вопрос)**:
1. Как влияет длина ответа на вероятность ответа при скоринге? Если есть какие-либо проблемы, как бы вы с этим боролись.
2. Если к началу каждого ответа добавилить метки A) B) C) D) станет ли модель отвечать лучше или хуже?
Стоит ли по-вашему добавлять эти метки?
